In [35]:
import pandas as pd

filepath = "https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx_errors.xlsx"

dfs = pd.read_excel(filepath, sheet_name=None)

# dfs.keys()

# dfs['header'].info()

dfs['header'].head()

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB
0,566910,2,27/03/2008,WROM,D051910639385,NaN,NaN
1,384923,2,09/09/2011,REFU,L034811502612,NaN,NaN
2,709901,1,14/02/2013,ABAN,Y021751958510,NaN,NaN
3,550084,2,04/05/2008,AOTH,R095578985099,NaN,NaN
4,710229,2,15/04/2002,REFU,Z095096287583,NaN,NaN


In [36]:
header = dfs['header']

# ssda903 102 - date of birth is not a valid date

# coercing errors returns NATs for invalid dates
# if I slice out all NaTs - I have all the invalid dates!
header['DOB_dt'] = pd.to_datetime(header['DOB'], format="%d/%m/%Y", errors='coerce')

# .isna() rewutnrs true where the value in a row in a column is an Na
invalid_dobs = header['DOB_dt'].isna()
error_rows = header[invalid_dobs]

error_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt
2675,bad_entry_1,7,oops!,Alien,UPDOG,NaN,NaN,NaT


In [37]:
# 115 - Date of Local Authority's (LA) decision that a child 
# should be placed for adoption is not a valid date."

# select the placed for adation table from dfs, 
p4a = dfs['placed_for_adoption']
# p4a.info()
# convert the correct column to a datetime, 
p4a['DATE_PLACED_dt'] = pd.to_datetime(p4a['DATE_PLACED'], format="%d/%m/%Y", errors='coerce')
# make the slice
error_rows = p4a[p4a['DATE_PLACED_dt'].isna()]

error_rows

,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,DATE_PLACED_dt
29,bad_entry_1,Not a date,This isn't a valid date,This also isn't a date,oop,NaT


In [38]:
# .isin() -> returns true where a column has a value in a list... it is in the list
# 103 - The ethnicity code is either not valid or has not been entered.

ethncity_codes = [
        "WBRI",
        "WIRI",
        "WOTH",
        "WIRT",
        "WROM",
        "MWBC",
        "MWBA",
        "MWAS",
        "MOTH",
        "AIND",
        "APKN",
        "ABAN",
        "AOTH",
        "BCRB",
        "BAFR",
        "BOTH",
        "CHNE",
        "OOTH",
        "REFU",
        "NOBT",
    ]
# ~ 
error_rows = header[~header['ETHNIC'].isin(ethncity_codes)]
error_rows



,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt
2675,bad_entry_1,7,oops!,Alien,UPDOG,NaN,NaN,NaT


In [39]:
# .isin()
# .astype() to change types to make sure we do selections right
# 114 - Data entry to record the status of former carer(s) of an adopted child is invalid.
# When we do an isin we need to check the datatype of the list is the same as the values
# valid codes 0, 1
# table is ad1
# FOSTER_CARE

ad1 = dfs['ad1'] 

ad1['FOSTER_CARE'] = ad1['FOSTER_CARE'].astype('str')

error_rows = ad1[~ad1['FOSTER_CARE'].isin(["0", "1"])]

error_rows

,CHILD,DOB,DATE_INT,DATE_MATCH,FOSTER_CARE,NB_ADOPTR,SEX_ADOPTR,LS_ADOPTR
29,bad_entry,datedate,notdate,datenot,777,Q,Will,let's go


In [40]:
# > <
# 184 - Date of decision that a child should be placed for adoption is before the child was born.
# return rows where DOB after DATE_PLACED -> DOB > DATE_PLACED

merged_df = p4a.merge(header[['CHILD', 'DOB_dt']], on='CHILD', how='left', suffixes=[None, '_head'])

error_rows = merged_df[merged_df["DOB_dt"] > merged_df['DATE_PLACED_dt']]

error_rows

# 302 - First episode starts before child was born.
# tables are header and episodes
# DOB from header and DECOM from episodes

episodes = dfs['episodes']

episodes['DECOM_dt'] = pd.to_datetime(episodes['DECOM'], dayfirst=True)

df = episodes.merge(header[['DOB_dt', 'CHILD']], how='left', on='CHILD')

error_rows = df[df['DECOM_dt'] < df['DOB_dt']]

# error_rows

In [41]:
# sort_values()
# drop_duplicates()

epi_ordered = episodes.sort_values(['CHILD', 'DECOM_dt'], ascending=True)

first_episode = epi_ordered.drop_duplicates('CHILD', keep='first')

first_episode

,CHILD,DECOM,RNE,LS,CIN,PLACE,PLACE_PROVIDER,DEC,REC,REASON_PLACE_CHANGE,HOME_POST,PL_POST,URN,DECOM_dt
4801,32,12/02/2017,T,J1,N3,K2,PR1,15/04/2017,E3,NaN,ED8 1OX,WI5 9CC,7876040,2017-02-12
4041,982,27/11/2016,P,E1,N7,A5,PR3,07/10/2017,X1,LAREQ,EP10 1XZ,WI5 8XU,6162294,2016-11-27
437,1243,02/01/2017,L,V4,N5,Z1,PR0,18/05/2017,E9,NaN,EC5 5MZ,WC30 9OS,7270736,2017-01-02
1794,1407,28/11/2015,L,L2,N3,P2,PR1,29/04/2017,X1,ALLEG,EG16 2ZL,WU19 1JS,6492318,2015-11-28
3848,2431,20/02/2016,S,V2,N3,P2,PR5,18/11/2017,X1,APPRR,EK12 3JI,WV1 1FJ,5227516,2016-02-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,997201,18/05/2016,P,L3,N4,T3,PR3,07/06/2017,X1,CUSTOD,EL12 3BA,WV22 6GC,3780837,2016-05-18
2766,997445,20/06/2016,P,J1,N2,U3,PR3,24/09/2017,E48,NaN,EP2 2VO,WE6 1FZ,3786993,2016-06-20
154,998192,27/11/2015,P,C2,N2,P1,PR5,08/07/2017,X1,ALLEG,EJ11 4FS,WT21 5RE,6722585,2015-11-27
156,998777,11/10/2015,P,C2,N2,U1,PR2,17/03/2018,X1,CLOSE,ER24 5VS,WJ7 5PQ,1112628,2015-10-11
